# Pairwise optimization of high-order interactions

This notebook demonstrates how we perform pair-to-pair optimization between
subjects belonging to two different conditions:
- a conscious group, and
- a non-responsive group.

For every possible pair of subjects (one from each group), we search for a set
of brain regions (an *n*-plet) that maximizes the difference in O-information
between the two subjects.


Note: This notebook should be run from the `high-order-anesthesia` folder to ensure the correct imports and file paths are used.

In [12]:
from pathlib import Path
import os
def ensure_project_root(target_name: str = "high-order-anesthesia") -> Path:
    cwd = Path.cwd().resolve()
    if cwd.name == target_name:
        return cwd
    for parent in cwd.parents:
        if parent.name == target_name:
            os.chdir(parent)
            return parent
    raise RuntimeError(
        f"Could not find '{target_name}' in current path or parents. "
        f"Please run the notebook from inside the project."
    )
ROOT = ensure_project_root("high-order-anesthesia")
print(f"Now in: {ROOT.name}")


Now in: high-order-anesthesia


### Overview of the optimization strategy

For each conscious / non-responsive subject pair, we optimize a score defined as
the **difference in O-information** between the two subjects.

We consider two complementary tasks:

- **Cpos**:  
  O-information(conscious) − O-information(non-responsive)  
  → identifies region sets where consciousness shows higher O-Information.

- **NRpos**:  
  O-information(non-responsive) − O-information(conscious)  
  → identifies region sets where consciousness shows lower O-Information.

This bidirectional analysis allows us to identify n-plets that exhibit the largest differences between states, while explicitly accounting for the
direction of change. In particular, it distinguishes whether an n-plet shifts toward more synergistic or more redundant interactions when  transitioning from the conscious to the non-responsive condition, or vice versa.


In [13]:
import pandas as pd
import torch
import numpy as np
import pandas as pd
import h5py
from collections import defaultdict
import time
import itertools
import logging
from tqdm.notebook import tqdm, trange

#### Custom libraries

In [14]:
%%capture
from src.hoi_anesthesia.thoi_utils import simulated_annealing_parallel
from src.hoi_anesthesia.utils import max_difference_pairs
from src.hoi_anesthesia.io import load_covariance_dict, print_time
from src.hoi_anesthesia.plotting import plot_measures_accross_states

#### Data loading and preparation

In [15]:
results_path = "results"
data_path = "data"

# Load covariance matrices
all_covs = load_covariance_dict(f"{data_path}/covariance_matrices_gc.h5")

# States for each dataset; MA: Multi-anesthesia - DBS: Deep Brain Stimulation
conscious_states = {
    "MA": ["MA_awake"],  
    "DBS": ["DBS_awake", "ts_on_5V"],
}
nonresponsive_states = {
    "MA": ["ts_selv2", "ts_selv4", "moderate_propofol", "deep_propofol", "ketamine"],
     "DBS": ["ts_off", "ts_on_3V_control", "ts_on_5V_control"],
}


#### Simulated Annealing parameters

The search over region combinations is performed using a modified version of
simulated annealing from the `thoi` library.

Key differences:
- Each subject pair is optimized independently
- Multiple pairs are optimized in parallel

The parameter `batch_size` controls how many subject pairs are optimized
simultaneously.


In [16]:
early_stop = 10
max_iter = 100
repeat = 10
batch_size = 300
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.empty_cache()

#### Select dataset and orders to optimize 

The optimization is repeated for multiple *orders* (i.e., number of regions in
the n-plet).

For each dataset and each order:
- all state combinations are processed,
- both optimization tasks (Cpos and NRpos) are evaluated,
- results are saved to disk as a CSV file.

A checkpoint CSV is written:
- after completing each state combination, and
- after completing each order.

This allows the computation to be safely resumed without losing previously computed results.


In [17]:
datasets_to_optimize = ['DBS', 'MA']
orders = [3,4,5]

for selected_dataset in datasets_to_optimize:
    t_i = time.time()
    for order in orders:
        results = []
        print("*" * 30)
        print("ORDER:", order)
        # Iterate over dataset/state combinations
        for state_c, state_nr in itertools.product(
            conscious_states[selected_dataset], nonresponsive_states[selected_dataset]
        ):
            covs_c = all_covs[selected_dataset][state_c]  # shape (N_c, 82, 82)
            covs_nr = all_covs[selected_dataset][state_nr]  # shape (N_nr, 82, 82)
            for target_task in ["Cpos", "NRpos"]:
                torch.cuda.empty_cache()
                cov_list = []
                subject_indices = []
                for i in range(covs_c.shape[0]):
                    for j in range(covs_nr.shape[0]):
                        cov_c = torch.from_numpy(covs_c[i])  # 82x82
                        cov_nr = torch.from_numpy(covs_nr[j])  # 82x82
                        if target_task == "Cpos":
                            cov_list.append(torch.stack([cov_c, cov_nr], dim=0))
                        elif target_task == "NRpos":
                            cov_list.append(torch.stack([cov_nr, cov_c], dim=0))
                        subject_indices.append([i, j])

                X = torch.stack(cov_list, dim=0).to(device)
                n_batches = X.shape[0] // batch_size + 1
                t_x = time.time()
                print(
                    f"Evaluating {state_c} vs {state_nr} with {X.shape[0]} pairs for task: {target_task}"
                )
                for idx in range(n_batches):
                    batched_X = X[idx * batch_size : (idx + 1) * batch_size, ...]
                    batched_sub_indices = subject_indices[
                        idx * batch_size : (idx + 1) * batch_size
                    ]
                    torch.cuda.empty_cache()
                    optimal_nplets, optimal_scores = simulated_annealing_parallel(
                        X=batched_X,
                        order=order,
                        device=device,
                        largest=True,
                        metric=max_difference_pairs,
                        repeat=repeat,
                        early_stop=early_stop,
                        max_iterations=max_iter,
                        covmat_precomputed=True,
                        batch_size=batch_size,
                        verbose=logging.WARNING,
                    )
                    max_idx = torch.argmax(optimal_scores, dim=0)  #

                    best_nplets = optimal_nplets[
                        max_idx, torch.arange(optimal_nplets.size(1))
                    ]
                    best_scores = optimal_scores[
                        max_idx, torch.arange(optimal_scores.size(1))
                    ]

                    for best_score, best_nplet, sub_indices in zip(
                        best_scores, best_nplets, batched_sub_indices
                    ):
                        best_nplet = best_nplet.tolist()
                        best_nplet.sort()
                        dataset_c = selected_dataset
                        dataset_nr = selected_dataset
                        subject_c, subject_nr = sub_indices

                        results.append(
                            {
                                "order": order,
                                "task": target_task,
                                "state_c": state_c,
                                "state_nr": state_nr,
                                "subject_c": subject_c,
                                "subject_nr": subject_nr,
                                "optimal_nplet": best_nplet,
                                "optimal_score": best_score.item(),
                            }
                        )
                    torch.cuda.empty_cache()

                results_df = pd.DataFrame(results)
                results_df.to_csv(
                    f"{results_path}/R1_A_max_O_diff_{selected_dataset}_{order}.csv",
                    index=False,
                    encoding="utf-8-sig",
                    sep=";",
                    decimal=",",
                )
                t_y = time.time()
                print(
                    f"{X.shape[0]} pairs evaluated in:",
                    np.round(t_y - t_x, 1),
                    "seconds",
                )
        results_df = pd.DataFrame(results)
        results_df.to_csv(
            f"{results_path}/R1_A_max_O_diff_{selected_dataset}_{order}.csv",
            index=False,
            encoding="utf-8-sig",
            sep=";",
            decimal=",",
        )


******************************
ORDER: 3
Evaluating DBS_awake vs ts_off with 1008 pairs for task: Cpos


  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.4337666786457154 - ES: 0: 100%|██████████| 100/100 [00:02<00:00, 37.64it/s]
  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.2471076203332001 - ES: 0: 100%|██████████| 100/100 [00:02<00:00, 37.81it/s]
  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.4194284340212072 - ES: 0: 100%|██████████| 100/100 [00:02<00:00, 36.30it/s]
  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.5145391999328767 - ES: 0: 100%|██████████| 100/100 [00:01<00:00, 61.94it/s]


1008 pairs evaluated in: 10.6 seconds
Evaluating DBS_awake vs ts_off with 1008 pairs for task: NRpos


  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.1370035852719959 - ES: 0: 100%|██████████| 100/100 [00:03<00:00, 32.85it/s] 
  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.12883668962336262 - ES: 0: 100%|██████████| 100/100 [00:02<00:00, 34.63it/s]
  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.2111016274657876 - ES: 0: 100%|██████████| 100/100 [00:02<00:00, 35.59it/s]
  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.19165459511080643 - ES: 0: 100%|██████████| 100/100 [00:01<00:00, 63.32it/s]


1008 pairs evaluated in: 11.2 seconds
Evaluating DBS_awake vs ts_on_3V_control with 648 pairs for task: Cpos


...

552 pairs evaluated in: 8.2 seconds
Evaluating MA_awake vs ketamine with 528 pairs for task: Cpos


  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.618222566499687 - ES: 0: 100%|██████████| 100/100 [00:03<00:00, 28.41it/s] 
  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.5807630253523192 - ES: 0: 100%|██████████| 100/100 [00:02<00:00, 38.95it/s]


528 pairs evaluated in: 6.6 seconds
Evaluating MA_awake vs ketamine with 528 pairs for task: NRpos


  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.28487630232568106 - ES: 0: 100%|██████████| 100/100 [00:03<00:00, 29.46it/s]
  0%|          | 0/100 [00:00<?, ?it/s]
mean(max_difference_pairs) = 0.26587009349306473 - ES: 0: 100%|██████████| 100/100 [00:02<00:00, 38.00it/s]


528 pairs evaluated in: 6.5 seconds


We can merge the results after the process is finished:

In [18]:
for selected_dataset in datasets_to_optimize:
    all_orders = pd.DataFrame()
    for order in orders:
        path = Path(results_path) / f"R1_A_max_O_diff_{selected_dataset}_{order}.csv"
        temp = pd.read_csv(path, sep=";", decimal=",", encoding="utf-8-sig")
        all_orders = pd.concat([all_orders, temp], ignore_index=True)
    
    all_orders.to_csv(
        f"{results_path}/R1_A_max_O_diff_{selected_dataset}_all_orders.csv",
        index=False,
        encoding="utf-8-sig",
        sep=";",
        decimal=",",
    )

### Output

For every subject pair and optimization task, we store:

- the n-plet order (number of regions),
- the task type (Cpos or NRpos),
- the conscious and non-responsive states,
- the indices of the paired subjects,
- the optimal n-plet (sorted list of regions),
- the associated optimization score.

In [19]:
all_orders.head(10)

,order,task,state_c,state_nr,subject_c,subject_nr,optimal_nplet,optimal_score
0,3,Cpos,MA_awake,ts_selv2,0,0,"[24, 28, 68]",0.436837
1,3,Cpos,MA_awake,ts_selv2,0,1,"[28, 39, 72]",0.574011
2,3,Cpos,MA_awake,ts_selv2,0,2,"[13, 28, 68]",0.336274
3,3,Cpos,MA_awake,ts_selv2,0,3,"[34, 38, 79]",0.550383
4,3,Cpos,MA_awake,ts_selv2,0,4,"[27, 40, 72]",0.468745
5,3,Cpos,MA_awake,ts_selv2,0,5,"[26, 33, 35]",0.519293
6,3,Cpos,MA_awake,ts_selv2,0,6,"[14, 55, 75]",0.616689
7,3,Cpos,MA_awake,ts_selv2,0,7,"[26, 27, 33]",0.367378
8,3,Cpos,MA_awake,ts_selv2,0,8,"[27, 33, 80]",0.379683
9,3,Cpos,MA_awake,ts_selv2,0,9,"[26, 29, 31]",0.458275
